In [30]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.sql import DataFrameWriter
import pandas as pd

db = "sm"


In [2]:
tweets_sdf = spark.sql("SELECT status_id, textTranslated AS text FROM " + db + ".twitter")
#tweets_sdf = spark.sql("SELECT status_id, text FROM " + db + ".twitter WHERE lang = 'en'")

In [3]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W", minTokenLength=4, 
                                toLowercase=True)
tokenized_sdf = regexTokenizer.transform(tweets_sdf)

In [5]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_words_sdf = remover.transform(tokenized_sdf)

In [6]:
cv = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=100, minDF=10.0)
cvmodel = cv.fit(filtered_words_sdf)

In [24]:
vocabulary_local = pd.DataFrame(cvmodel.vocabulary, columns = ["word"])
vocabulary_local["word_id"] = vocabulary_local.index
vocabulary_sdf = spark.createDataFrame(vocabulary_local)
vocabulary_sdf.head()
vocabulary_sdf.createOrReplaceTempView("vocab_v")
spark.sql("CREATE TABLE " + db + ".vocab AS SELECT * FROM vocab_v")

DataFrame[]

In [28]:
spark.sql("SELECT * FROM " + db + ".vocab").toPandas()

,word,word_id
0,https,0
1,islam,1
2,immigration,2
3,muslim,3
4,migration,4
5,allah,5
6,quran,6
7,trump,7
8,ramadan,8
9,muslims,9


In [33]:
result = cvmodel.transform(filtered_words_sdf)
result.head(3)

[Row(status_id=u'972175221481459712', text=u'When your wonderful friend is on a trip around the world to literally help save the planet and you\u2018re at home watching cat videos all day... \U0001f602 #goSarah! @GreenpeaceAT @SarahSchrf https://t.co/h3CZbdEO5R', words=[u'when', u'your', u'wonderful', u'friend', u'trip', u'around', u'world', u'literally', u'help', u'save', u'planet', u'home', u'watching', u'videos', u'gosarah', u'greenpeaceat', u'sarahschrf', u'https', u'h3czbdeo5r'], filtered=[u'wonderful', u'friend', u'trip', u'around', u'world', u'literally', u'help', u'save', u'planet', u'home', u'watching', u'videos', u'gosarah', u'greenpeaceat', u'sarahschrf', u'https', u'h3czbdeo5r'], features=SparseVector(100, {0: 1.0, 19: 1.0, 49: 1.0})),
 Row(status_id=u'972082111606939648', text=u'@GrandHuit Rettet die Pinguine! #SaveAntarctic https://t.co/p7Dk438z3R', words=[u'grandhuit', u'rettet', u'pinguine', u'saveantarctic', u'https', u'p7dk438z3r'], filtered=[u'grandhuit', u'rettet', 

In [ ]:
result.write.saveAsTable(db + ".words")